In [9]:
import pandas as pd
import yfinance as yf

## Calculation for 2a.
# Create a DataFrame with ETF Tickers

data = {
'ETF Ticker': ['SPY',
'IWM', 'DIA']
}

df = pd.DataFrame(data)

# Remove the column name
df.columns = [None]

# Print the DataFrame
print(df.to_string(index=False,
header=False))

SPY
IWM
DIA


In [10]:
## Calculation for 2b. 

# Define ETF tickers and time period
etf_tickers = ['SPY','IWM', 'DIA']

start_date = '2023-01-01'
end_date = '2024-01-01'

# Fetch ETF prices
etf_prices = yf.download(etf_tickers,
start=start_date,
end=end_date)['Adj Close']

# Ensure columns are in the desired order
etf_prices = etf_prices[etf_tickers]

# Calculate portfolio return with equal weights
num_etfs = len(etf_tickers)

portfolio_weights = [1/num_etfs]* num_etfs

# Calculate portfolio return
portfolio_return = (etf_prices.pct_change()
* portfolio_weights).sum(axis=1)

# Calculate correlation of portfolio return with each ETF return
correlation_against_etf = etf_prices.pct_change().corrwith(portfolio_return)

# Create a DataFrame to display correlations against ETFs
correlation_table = pd.DataFrame(correlation_against_etf,
columns=['Correlation against ETF'])

print(correlation_table)

c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]

        Correlation against ETF
Ticker                         
SPY                    0.939430
IWM                    0.942948
DIA                    0.929564


In [11]:

## Calculation for 2c.
# Calculate returns
returns = etf_prices.pct_change()

# Calculate covariance matrix
covariance_matrix = returns.cov()

# Calculate covariance of the portfolio against each ETF
# Assuming equal portfolio weights
portfolio_weights = [1/len(etf_tickers)]* len(etf_tickers)

covariance_portfolio = (covariance_matrix * portfolio_weights).sum(axis=1)

# Create a DataFrame for both correlation and covariance against ETFs
portfolio_etfs = pd.DataFrame({'Correlation against ETF': correlation_against_etf, 'Covariance against ETF': covariance_portfolio}, index=etf_tickers)

print("\nCorrelation and Covariance against ETF:")
print(portfolio_etfs)


Correlation and Covariance against ETF:
     Correlation against ETF  Covariance against ETF
SPY                 0.939430                0.000068
IWM                 0.942948                0.000105
DIA                 0.929564                0.000059


In [12]:
## Calculation for 2d.
# Calculate tracking errors using trailing 10 years
tracking_errors = [(portfolio_return - returns[etf]).std()
for etf in etf_tickers]

# Create a DataFrame for tracking errors
tracking_errors_table = pd.DataFrame({'Tracking Errors':tracking_errors}, index=etf_tickers)

# Combine correlation, covariance, and tracking errors into one DataFrame
combined_table = pd.concat([portfolio_etfs,tracking_errors_table],axis=1)

# Display the final DataFrame
print("Combined Portfolio ETF Metrics Table:")
print(combined_table)


Combined Portfolio ETF Metrics Table:
     Correlation against ETF  Covariance against ETF  Tracking Errors
SPY                 0.939430                0.000068         0.003014
IWM                 0.942948                0.000105         0.005259
DIA                 0.929564                0.000059         0.003384


In [13]:
## Calculation for 2e.
# Calculate Sharpe Ratio using the current risk-free rate
risk_free_rate = 0.0427

sharpe_ratios = [(returns[etf].mean()- risk_free_rate) / returns[etf].std() for etf in etf_tickers]

# Create a DataFrame for Sharpe Ratios
sharpe_ratio_table = pd.DataFrame({'Sharpe Ratio' : sharpe_ratios}, index=etf_tickers)

# Combine correlation, covariance, tracking errors, and Sharpe Ratios into one DataFrame
combined_table = pd.concat([portfolio_etfs, tracking_errors_table, sharpe_ratio_table],axis=1)

# Display the final DataFrame
print("Combined Portfolio ETF Metrics Table:")
print(combined_table)

Combined Portfolio ETF Metrics Table:
     Correlation against ETF  Covariance against ETF  Tracking Errors  \
SPY                 0.939430                0.000068         0.003014   
IWM                 0.942948                0.000105         0.005259   
DIA                 0.929564                0.000059         0.003384   

     Sharpe Ratio  
SPY     -5.058298  
IWM     -3.315331  
DIA     -5.842461  


In [14]:
## Calculation for 2f.

import numpy as np

# Convert portfolio_weights to a NumPy array
portfolio_weights_array = np.array(portfolio_weights)

# Calculate Annualized Volatility (252 days) for both portfolio and ETFs
portfolio_volatility = np.sqrt(np.sum(portfolio_weights_array.T.dot(covariance_matrix).dot(portfolio_weights_array))* 252)
etf_volatility = np.sqrt(np.diag(covariance_matrix)* 252)

# Calculate Volatility Spread
volatility_spread = portfolio_volatility - etf_volatility

# Create a DataFrame for Volatility Spread
volatility_spread_table = pd.DataFrame({'Volatility Spread': volatility_spread}, index=etf_tickers)

# Combine correlation, covariance, tracking errors, Sharpe Ratios, and Volatility Spread into one DataFrame
combined_table = pd.concat([portfolio_etfs, tracking_errors_table,sharpe_ratio_table, volatility_spread_table], axis=1)

# Display the final DataFrame
print("Combined Portfolio ETF Metrics Table:")
print(combined_table)

Combined Portfolio ETF Metrics Table:
     Correlation against ETF  Covariance against ETF  Tracking Errors  \
SPY                 0.939430                0.000068         0.003014   
IWM                 0.942948                0.000105         0.005259   
DIA                 0.929564                0.000059         0.003384   

     Sharpe Ratio  Volatility Spread  
SPY     -5.058298           0.008675  
IWM     -3.315331          -0.061388  
DIA     -5.842461           0.025265  
